# Labeling the attributes of the 2011, 2013, 2016 and 2022 json files

This notebook uses the NLP models of Spacy, the country-converter, a dataset with cities and some other methods to label the attributes of the json files.

### Import Libraries and download the English transformer model from Spacy

In [1]:
# Libraries
import pandas as pd
import json
import country_converter as coco
import spacy
from spacy import displacy

In [2]:
#!python3 -m spacy download en_core_web_trf

### Function to label the attributes

We label the attributes in the following sequence:
- Spacy's NLP model classifies the attributes
- country: The Country-Converter detects countries and changes their name to a standard
- city: The attributes will be compared to a dataset of the biggest cities
- org: Organisations with a typical ending for companies (gmbh, ag, ...) will be identified
- dep: Typical company departments are detected.
- other: Some unlabeled attributes remain.

In [3]:
# Load Spacy English language model (outside of the function, so it does not have to be loaded everytime the function is called)
NER = spacy.load("en_core_web_trf")


def label_data(network_file_path):

    # Read the JSON-file
    with open(network_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    # List to store labeled data
    labeled_data = []

    # Process each attribute in the JSON file
    for key, values in data.items():
        for value in values:
            doc = NER(value)
            entities = ', '.join([ent.label_ for ent in doc.ents])  # Convert the list of entities to a string
            entities = entities.lower()
            labeled_data.append({'company': key, 'attribute': value, 'label': entities})

    # Convert list to DataFrame
    labeled_data_df = pd.DataFrame(labeled_data)

    # Read the cities and change them to lowercase
    with open('all_cities.txt', 'r') as file:
        cities = [city.strip().lower() for city in file.readlines()]

    # List with typical company endings
    endings = [' gmbh', ' sa', ' ltd', ' limited', ' ag', ' group']

    # List with relevant departments (some of them are from https://simplicable.com/en/company-departments)
    departments = ['compliance', 'corporate communication', 'employee relations', 'employee services', 'executive management', 'executive support',
                'facilities management', 'accounting', 'human resources', 'internal audit', 'legal affairs', 'media relations', 'office management',
                'quality assurance', 'reception services', 'recruitment', 'risk management', 'advertising', 'brand management', 'creative services', 
                'customer engagement', 'marketing', 'product development', 'product management', 'public relations', 'sales', 'customer service', 
                'distribution', 'inventory management', 'logistics', 'maintenance', 'operations management', 'procurement', 'purchasing', 'production',
                'project management', 'quality control', 'supplier management', 'supply chain management', 'information technology']

    # Check each row and update the labels
    for index, row in labeled_data_df.iterrows():
        
        # Try to convert the attribute to name_short
        try:
            converted = coco.convert(names=row['attribute'], to='name_short')

            # If the Conversion was successful, update the attribute and change the label to 'country'
            if converted != 'not found':
                labeled_data_df.at[index, 'attribute'] = converted.lower()
                labeled_data_df.at[index, 'label'] = 'country'
                continue    # go to the next attribute

        # If the Conversion was not successful, check for city, company and department
        except Exception:
            pass

        # Check if the attribute is a city and update the label
        if row['attribute'] in cities:
            labeled_data_df.at[index, 'label'] = 'city'
        
        # Check if the attribute ends with one of the typical company endings; in this case, change the label to 'org'
        elif any(row['attribute'].endswith(end) for end in endings):
            labeled_data_df.at[index, 'label'] = 'org'

        # Check if the attribute contains one of the department names and update the label only if it was empty before
        elif any(department in row['attribute'] for department in departments) and row['label'] == '':
            labeled_data_df.at[index, 'label'] = 'dep'

        # Check if the label is empty or there are more labels than one for this attribute (this is the case when spacy's result were two or more labels for one attribute)
        elif pd.isna(row['label']) or row['label'] == '' or row['label'] is None or ',' in str(row['label']):
            labeled_data_df.at[index, 'label'] = 'other'
        
    # Export the labeled data as csv
    output_file_name = 'labeled_' + network_file_path.split('.')[0] + '.csv'
    labeled_data_df.to_csv(output_file_name, index=False)

#### Apply the function to the 2011 file

In [4]:
#label_data('../Files/BDO_uploads/network_2011_update_28112023.txt')

#### Apply the function to the 2013 file

In [5]:
#label_data('../Files/BDO_uploads/network_2011_update_28112023.txt')

#### Apply the function to the 2016 file

In [6]:
#label_data('../Files/BDO_uploads/network_2011_update_28112023.txt')

#### Apply the function to the 2022 file

In [7]:
#label_data('../Files/BDO_uploads/network_2011_update_28112023.txt')

# Improvement of the labeling algorithm

The function above takes a long for labeling the datasets. Our further work is therefore based on the already labeled data. In this section we attempt to further refine the classification.

In [8]:
# Read the previously labeled csv files
testfile_2011 = pd.read_csv('labeled_network_2011_TEST.csv')
labeled_2011 = pd.read_csv('labeled_network_2011_update_28112023.csv')
labeled_2013 = pd.read_csv('labeled_network_2013_update_28112023.csv')
labeled_2016 = pd.read_csv('labeled_network_2016_update_28112023.csv')
labeled_2022 = pd.read_csv('labeled_network_2022_update_28112023.csv')

### Use the German transformer model from Spacy

As suggested by BDO, we try to use the German transformer model from Spacy instead of the English one. Before labeling the whole data again - which takes a long time - we apply this model on a smaller test set with with three Austrian companies Raiffeisen Bank International, Erste Bank and Andritz.

In [9]:
#!python3 -m spacy download de_dep_news_trf

In [10]:
# Load Spacy German language model
NER_de = spacy.load("de_dep_news_trf")

# Function to label the text using the German transformer model
def label_text_de(text):
    doc = NER_de(text)
    if doc.ents:  # if the model found entities
        return doc.ents[0].label_  # return the label of the first entity
    else:
        return "other"  # if no entity found, return 'other'

# Apply the function to the 'attribute' column to create the new 'label_de' column
testfile_2011_de = testfile_2011
testfile_2011_de['label_de'] = testfile_2011_de['attribute'].apply(label_text_de)

In [11]:
# Define a function for printing a full dataframe
def print_full(df):
    pd.set_option('display.max_rows', len(df))
    print(df)
    pd.reset_option('display.max_rows')

In [12]:
print_full(testfile_2011_de)

                               company  \
0     raiffeisen bank international ag   
1     raiffeisen bank international ag   
2     raiffeisen bank international ag   
3     raiffeisen bank international ag   
4     raiffeisen bank international ag   
5     raiffeisen bank international ag   
6     raiffeisen bank international ag   
7     raiffeisen bank international ag   
8     raiffeisen bank international ag   
9     raiffeisen bank international ag   
10    raiffeisen bank international ag   
11    raiffeisen bank international ag   
12    raiffeisen bank international ag   
13    raiffeisen bank international ag   
14    raiffeisen bank international ag   
15    raiffeisen bank international ag   
16    raiffeisen bank international ag   
17    raiffeisen bank international ag   
18    raiffeisen bank international ag   
19    raiffeisen bank international ag   
20    raiffeisen bank international ag   
21    raiffeisen bank international ag   
22    raiffeisen bank internationa

As we notice at the dataframe above, the German Spacy model could not label even one single attribute out of 1.500 representing the three Austrian companies. We therefore assume that it is not expedient to use this model or models in other languages.

### Use lists of first and last names from data.europa.eu to filter the attributes labeled as person

This next method was suggested by Ben Gronemann. Here, we use lists of first and last names from data.europe.eu in order to improve the accuracy of labeling persons. Additionally, we check for common words between the company and the attribute to ensure that attributes like for example 'zao raiffeisen' are not labeled as persons and we check for ' ' in the attribute, because a single word cannot not be a person. We assume that a person has to be named with first and last name.

In [13]:
# Function to update the label 'person' - FIRST AND LAST NAME
def update_label_first_and_last(row, first_names_list, last_names_list):
    attribute = str(row['attribute']) 
    company = str(row['company'])

    # Check for common words between company and attribute
    if any(company_part in attribute for company_part in company.split()):
        return 'other'
        
    # Set the booleans to False
    first_name = False
    last_name = False
    
    # Check if the attribute contains a first name
    for name in first_names_list:
        if name in attribute and ' ' in attribute:
            first_name = True
            break

    # Check if the attribute contains a last name
    for name in last_names_list:
        if name in attribute:
            last_name = True
            break

    if first_name and last_name:
        return row['label']  # Return the original label if a name is found

    return 'other'  # Label as 'other' if no name is found in attribute


# Function to update the label 'person' - FIRST NAME ONLY
def update_label_only_first(row, names_list):
    attribute = str(row['attribute'])
    company = str(row['company'])

    # Check for common words between company and attribute
    if any(company_part in attribute for company_part in company.split()):
        return 'other'

    # Check for names in attribute
    for name in names_list:
        if str(name).lower() in attribute.lower() and ' ' in attribute:
            return row['label']  # Return the original label if a name is found
    return 'other'  # Label as 'other' if no name is found in attribute


# Function to extract person names from the attribute
NER = spacy.load("en_core_web_trf")
def extract_person(attribute):
    doc = NER(attribute)
    person_parts = [ent.text for ent in doc.ents]
    return ' '.join(person_parts)


def improve_person_labeling(labeled_network_df, onlyfirst_or_firstandlast = 'onlyfirst'):
    
    # Read the csv files with first and last names
    first_names = pd.read_csv('Liste der Vornamen-1.csv')
    last_names = pd.read_csv('Liste der Nachnamen-2.csv')

    # Change the 'prenom'/'patronyme' column to lowercase
    first_names.loc[:, 'prenom'] = first_names['prenom'].str.lower()
    last_names.loc[:, 'patronyme'] = last_names['patronyme'].str.lower()

    # Filter the dataframes
    first_names_filtered = first_names[first_names['sum'] > 350]
    last_names_filtered = last_names[last_names['count'] > 100]

    # Convert the names dataframes to lists
    first_names_list = first_names_filtered['prenom'].tolist()
    last_names_list = last_names_filtered['patronyme'].tolist()

    # Split the original network dataframe into two parts
    network_person = labeled_network_df[labeled_network_df['label'] == 'person']
    network_without_person = labeled_network_df[labeled_network_df['label'] != 'person']

    # Update the 'attribute' column by applying the function to extract persons
    network_person = network_person.copy()  # Create a copy of the slice to ensure it's independent from the original dataframe (suggested by Warning message)
    network_person.loc[network_person.index, 'attribute'] = network_person['attribute'].apply(extract_person)

    if onlyfirst_or_firstandlast == 'firstandlast':
        # Apply the function to update the labels of the person-df
        network_person.loc[:, 'label'] = network_person.apply(update_label_first_and_last, axis=1, args=(first_names_list, last_names_list,))

    elif onlyfirst_or_firstandlast == 'onlyfirst':
        # Apply the function to update the labels of the person-df
        network_person.loc[:, 'label'] = network_person.apply(update_label_only_first, axis=1, args=(first_names_list,))

    else:
        return 'Please enter "firstandlast" or "onlyfirst" as the second argument'

    # Put the two dataframes together
    updated_df = pd.concat([network_person, network_without_person], ignore_index=True)

    return updated_df


After testing the function with different parameters on the testfile, we concluded that the best result is delivered by using both, the first (count > 350) and the last (count > 100) name. 

In [14]:
# Apply the function on the testfile to check the accuracy
testfile_updated = improve_person_labeling(testfile_2011, 'firstandlast')
print_full(testfile_updated)

                               company  \
0     raiffeisen bank international ag   
1     raiffeisen bank international ag   
2     raiffeisen bank international ag   
3     raiffeisen bank international ag   
4     raiffeisen bank international ag   
5     raiffeisen bank international ag   
6     raiffeisen bank international ag   
7     raiffeisen bank international ag   
8     raiffeisen bank international ag   
9     raiffeisen bank international ag   
10    raiffeisen bank international ag   
11    raiffeisen bank international ag   
12    raiffeisen bank international ag   
13    raiffeisen bank international ag   
14    raiffeisen bank international ag   
15    raiffeisen bank international ag   
16    raiffeisen bank international ag   
17    raiffeisen bank international ag   
18    raiffeisen bank international ag   
19    raiffeisen bank international ag   
20    raiffeisen bank international ag   
21    raiffeisen bank international ag   
22    raiffeisen bank internationa

In [15]:
# Apply the function on all network dataframes to update the 'person' labels
labeled_2011_updated = improve_person_labeling(labeled_2011, 'firstandlast')
labeled_2013_updated = improve_person_labeling(labeled_2013, 'firstandlast')
labeled_2016_updated = improve_person_labeling(labeled_2016, 'firstandlast')
labeled_2022_updated = improve_person_labeling(labeled_2022, 'firstandlast')

In [65]:
# Function to manually label some attributes right
def update_labels_manually(labeled_network_df):

    # Define the mapping from attribute to new label
    label_mapping = {
        'são paulo': 'city',
        'monte carlo': 'gpe',
        'jp morgan': 'org',
        'morgan stanley': 'org',
        'eli lilly': 'org',
        'dow jones': 'other',
        'boehringer ingelheim': 'other',
        'imd lausanne': 'dep',
        'são paulo brl': 'other',
        'deloitte touche': 'other',
        'akzo nobel': 'other',
        'srl milan': 'org',
        'frost sullivan': 'other',
        'euro pean': 'gpe',
        'lehman brors': 'org',
        'sl madrid': 'other',
        'lenz staehelin': 'org', 
        'kepler cheuvreux': 'org',
        'ihs markit': 'org',
        'georg fischer': 'org',
        'barry callebaut': 'org',
        'chartder vielfalt': 'other',
        'arthur anderson': 'org'
    }

    # Apply the mapping to the dataframe
    for attribute, new_label in label_mapping.items():
        labeled_network_df.loc[labeled_network_df['attribute'] == attribute, 'label'] = new_label

    return labeled_network_df

In [66]:
# Apply the function on all network dataframes
labeled_2011_updated = update_labels_manually(labeled_2011_updated)
labeled_2013_updated = update_labels_manually(labeled_2013_updated)
labeled_2016_updated = update_labels_manually(labeled_2016_updated)
labeled_2022_updated = update_labels_manually(labeled_2022_updated)

# Load and Preprocess the CIQ data

In [67]:
import numpy as np

# Read the 2023 CIQ_data
df_ciq = pd.read_csv('../Files/CIQ_data_DS_LAB.csv')

# Choose the relevant columns and change them to lowercase
df_ciq = df_ciq[['Name', 'IQ_PRIMARY_INDUSTRY', 'IQ_EXCHANGE', 'IQ_CITY', 'IQ_COUNTRY_NAME', 'IQ_EMPLOYEES', 'IQ_CEO_NAME', 'IQ_CFO_NAME', 'IQ_SP_LC_LT', 
                'Supplier1', 'Supplier2', 'Supplier3', 'Supplier4', 'Supplier5', 'Supplier6', 'Supplier7', 'Supplier8', 'Supplier9', 'Supplier10',
                'customer1', 'customer2', 'customer3', 'customer4', 'customer5', 'customer6', 'customer7', 'customer8', 'customer9', 'customer10',
                'bm1', 'bm2', 'bm3', 'bm4', 'bm5', 'bm6', 'bm7', 'bm8', 'bm9', 'bm10']].apply(lambda x: x.str.lower())

# Drop rows with a specific 'Name'
df_ciq = df_ciq[~df_ciq['Name'].isin(['Nordea Bank Abp', 'Dassault Aviation SA', '1 - 600 of 600'])]

# Converting 'Board Member' entries in 'bm2' to NaN
df_ciq['bm2'] = df_ciq['bm2'].replace('Board Member', np.nan)

# Rename the headers
df_ciq.rename(columns={'IQ_PRIMARY_INDUSTRY': 'industry'}, inplace=True)
df_ciq.rename(columns={'IQ_EXCHANGE': 'exchange'}, inplace=True)
df_ciq.rename(columns={'IQ_CITY': 'home_city'}, inplace=True)
df_ciq.rename(columns={'IQ_COUNTRY_NAME': 'home_country'}, inplace=True)
df_ciq.rename(columns={'IQ_EMPLOYEES': 'employees'}, inplace=True)
df_ciq.rename(columns={'IQ_CEO_NAME': 'ceo'}, inplace=True)
df_ciq.rename(columns={'IQ_CFO_NAME': 'cfo'}, inplace=True)
df_ciq.rename(columns={'IQ_SP_LC_LT': 'rating'}, inplace=True)

# Change the column headers of the company_df to lowercase
df_ciq.columns = [col.lower() for col in df_ciq.columns]

# Replace '(invalid identifier)' with NaN
df_ciq = df_ciq.replace(to_replace='(invalid identifier)', value=np.nan)

# Replace the values in the 'industry' column
df_ciq['industry'] = df_ciq['industry'].replace({'apparel, accessories and luxury goods': 'apparel accessories and luxury goods',
                                                        'multi-line insurance': 'multi_line insurance'})

# Person and org columns to remove commas from
columns_to_remove_commas = ['ceo', 'cfo', 'supplier1', 'supplier2', 'supplier3', 'supplier4', 'supplier5', 'supplier6', 'supplier7', 'supplier8', 'supplier9', 'supplier10',
                            'customer1', 'customer2', 'customer3', 'customer4', 'customer5', 'customer6', 'customer7', 'customer8', 'customer9', 'customer10',
                            'bm1', 'bm2', 'bm3', 'bm4', 'bm5', 'bm6', 'bm7', 'bm8', 'bm9', 'bm10']

# Apply the replace function to each specified column
for col in columns_to_remove_commas:
    df_ciq[col] = df_ciq[col].str.replace(',', '')


In [68]:
def convert_ratings_to_numeric(dataframe, column_name, conversion_dict):

    # Check if the column exists in the dataframe
    if column_name not in dataframe.columns:
        raise ValueError(f"The column {column_name} does not exist in the dataframe.")
    
    # Replace the ratings using the conversion dictionary
    dataframe[column_name] = dataframe[column_name].map(conversion_dict)
    
    # Handle any ratings that could not be converted (if any)
    if dataframe[column_name].isnull().any():
        missing_ratings = dataframe[column_name].isnull().sum()
        print(f"Warning: {missing_ratings} ratings could not be converted and are set as NaN.")
    
    return dataframe

conversion_dict = {
    'aaa': 100,
    'aa+': 95,
    'aa': 90,
    'aa-': 85,
    'a+': 80,
    'a': 75,
    'a-': 70,
    'bbb+': 65,
    'bbb': 60,
    'bbb-': 55,
    'bb+': 50,
    'bb': 45,
    'bb-': 40,
    'b+': 35,
    'b': 30,
    'b-': 25,
    'ccc+': 20,
    'ccc': 20,
    'ccc-': 20,
    'cc': 15,
    'c': 10,
    'd': 5
}

df_ciq = convert_ratings_to_numeric(df_ciq, 'rating', conversion_dict)

In [69]:
# Use the country_converter to convert to name_short
df_ciq['home_country'] = coco.convert(names=df_ciq['home_country'], to='name_short')
df_ciq[['home_country']] = df_ciq[['home_country']].apply(lambda x: x.str.lower())

nan not found in ISO3
nan not found in ISO3


In [70]:
# Save the preprocessed ciq file as csv
df_ciq.to_csv('CIQ_data_DS_LAB_Preprocessed.csv', index=False)

# Merge industry, exchange, city and country from 2023 CIQ data and financial data to the network files

In [71]:
# Read previously saved and preprocessed 2023 ciq data
df_ciq = pd.read_csv('CIQ_data_DS_LAB_Preprocessed.csv')

In [72]:
def ciq_data_into_network_format(labeled_network_file, ciq_data):

    # Initialize an empty list for the new rows
    new_rows = []

    # Specify the columns to extract from df_ciq - we assume that those four attributes do not change over time
    columns_to_extract = ['home_city', 'home_country', 'industry', 'exchange']

    # Get a list of unique companies from labeled_network_file
    companies_in_networkfile = labeled_network_file['company'].unique()

    # Iterate over all rows in df_ciq
    for _, row in df_ciq.iterrows():
        company_name = row['name']
        # Check if the company is in labeled_network_file
        if company_name in companies_in_networkfile:
            for col in columns_to_extract:
                # Skip the 'name' column as it is used for the company_name
                if col != 'name':
                    # Check if the column value is not NaN
                    if pd.notna(row[col]):
                        new_row = {
                            'company': company_name,
                            'attribute': f"{row[col]}",
                            'label': col
                        }
                        new_rows.append(new_row)

    # Convert the list of dictionaries to a DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    return new_rows_df


In [73]:
# We assume that ceo, cfo, customer, supplier and boardmember did not change much over one year.
# Therefore, these attributes are additionally merged to the 2022 network file.
def ciq_data_into_network_format_2022(labeled_network_file, ciq_data):

    # Initialize an empty list for the new rows
    new_rows = []

    # Get a list of unique companies from labeled_network_file
    companies_in_networkfile = labeled_network_file['company'].unique()

    # Iterate over all rows in df_ciq
    for _, row in df_ciq.iterrows():
        company_name = row['name']
        # Check if the company is in labeled_network_file
        if company_name in companies_in_networkfile:
            for col in df_ciq.columns:
                # Skip the 'name' column as it is used for the company_name
                if col != 'name':
                    # Check if the column value is not NaN
                    if pd.notna(row[col]):
                        new_row = {
                            'company': company_name,
                            'attribute': f"{row[col]}",
                            'label': col
                        }
                        new_rows.append(new_row)

    # Convert the list of dictionaries to a DataFrame
    new_rows_df = pd.DataFrame(new_rows)

    # Update the 'label' column where it contains 'bm', 'supplier' or 'customer' 
    new_rows_df.loc[new_rows_df['label'].str.contains('bm'), 'label'] = 'bm'
    new_rows_df.loc[new_rows_df['label'].str.contains('supplier'), 'label'] = 'supplier'
    new_rows_df.loc[new_rows_df['label'].str.contains('customer'), 'label'] = 'customer'

    return new_rows_df

In [74]:
# Apply the ciq_data_into_network_format function for 2011, 2013, 2016 and 2022
ciq_2011 = ciq_data_into_network_format(labeled_2011_updated, df_ciq)
ciq_2013 = ciq_data_into_network_format(labeled_2013_updated, df_ciq)
ciq_2016 = ciq_data_into_network_format(labeled_2016_updated, df_ciq)
ciq_2022 = ciq_data_into_network_format_2022(labeled_2022_updated, df_ciq)

In [2]:
# Read the financial data
fin_2011 = pd.read_excel('../Files/BDO_uploads/CIQ_data_2011.xlsx')
fin_2013 = pd.read_excel('../Files/BDO_uploads/CIQ_data_2013.xlsx')
fin_2016 = pd.read_excel('../Files/BDO_uploads/CIQ_data_2016.xlsx')
fin_2022 = pd.read_excel('../Files/BDO_uploads/CIQ_data_2022.xlsx')

In [76]:
def financial_data_into_network_format(labeled_network_file, financial_data):

    # Change the column names to lowercase
    financial_data.columns = [col.lower() for col in financial_data.columns]

    # Drop the 'isin' column
    financial_data = financial_data.drop('isin', axis=1)

    # Change the 'name' column to lowercase
    financial_data[['name']] = financial_data[['name']].apply(lambda x: x.str.lower())

    # Initialize an empty list for the new rows
    new_rows_fin = []

    # Get a list of unique companies from labeled_network_file
    companies_in_networkfile = labeled_network_file['company'].unique()

    # Iterate over all rows in financial_data
    for _, row in financial_data.iterrows():
        company_name = row['name']
        # Check if the company is in testfile_2011
        if company_name in companies_in_networkfile:
            for col in financial_data.columns:
                # Skip the 'name' column as it is used for the company_name
                if col != 'name':
                    # Check if the column value is not NaN
                    if pd.notna(row[col]):
                        new_row = {
                            'company': company_name,
                            'attribute': row[col],
                            'label': col
                        }
                        new_rows_fin.append(new_row)

    # Convert the list of dictionaries to a DataFrame
    new_rows_fin_df = pd.DataFrame(new_rows_fin)

    return new_rows_fin_df

In [77]:
# Apply the financial_data_into_network_format function for 2011, 2013, 2016 and 2022
fin_2011_update = financial_data_into_network_format(labeled_2011_updated, fin_2011)
fin_2013_update = financial_data_into_network_format(labeled_2013_updated, fin_2013)
fin_2016_update = financial_data_into_network_format(labeled_2016_updated, fin_2016)
fin_2022_update = financial_data_into_network_format(labeled_2022_updated, fin_2022)

In [78]:
# Concatenate the 2023 ciq data and financial data to the network files
network_2011_updated = pd.concat([labeled_2011_updated, ciq_2011, fin_2011_update], ignore_index=True)
network_2013_updated = pd.concat([labeled_2013_updated, ciq_2013, fin_2013_update], ignore_index=True)
network_2016_updated = pd.concat([labeled_2016_updated, ciq_2016, fin_2016_update], ignore_index=True)
network_2022_updated = pd.concat([labeled_2022_updated, ciq_2022, fin_2022_update], ignore_index=True)

In [79]:
def sort_network_files(labeled_network_file):

    # Define a new order of columns
    new_order = ['company', 'label', 'attribute']

    # Reorder the columns
    labeled_network_file = labeled_network_file[new_order]

    # Sort the dataframe by company and label
    labeled_network_file = labeled_network_file.sort_values(by=['company', 'label'])

    return labeled_network_file

In [80]:
def filter_network_files_dach(labeled_network_file):

    # Filtering to DACH region
    # Find companies that meet the criteria
    criteria = (labeled_network_file['label'] == 'home_country') & labeled_network_file['attribute'].isin(['austria', 'germany', 'switzerland'])
    companies_meeting_criteria = labeled_network_file[criteria]['company'].unique()

    # Filter the DataFrame to include only these companies
    updated_network_file = labeled_network_file[labeled_network_file['company'].isin(companies_meeting_criteria)]

    return updated_network_file

In [81]:
# Apply the sort_network_files function for 2011, 2013, 2016 and 2022
network_2011_all = sort_network_files(network_2011_updated)
network_2013_all = sort_network_files(network_2013_updated)
network_2016_all = sort_network_files(network_2016_updated)
network_2022_all = sort_network_files(network_2022_updated)

In [82]:
# Apply the filter_network_files_dach function for 2011, 2013, 2016 and 2022
network_2011_dach = filter_network_files_dach(network_2011_all)
network_2013_dach = filter_network_files_dach(network_2013_all)
network_2016_dach = filter_network_files_dach(network_2016_all)
network_2022_dach = filter_network_files_dach(network_2022_all)

In [85]:
# Save the updated network files as csv - ALL
network_2011_all.to_csv('labeled_network_2011_20240114.csv', index=False)
network_2013_all.to_csv('labeled_network_2013_20240114.csv', index=False)
network_2016_all.to_csv('labeled_network_2016_20240114.csv', index=False)
network_2022_all.to_csv('labeled_network_2022_20240114.csv', index=False)

In [86]:
# Save the updated network files as csv - DACH
network_2011_dach.to_csv('labeled_network_2011_dach_20240114.csv', index=False)
network_2013_dach.to_csv('labeled_network_2013_dach_20240114.csv', index=False)
network_2016_dach.to_csv('labeled_network_2016_dach_20240114.csv', index=False)
network_2022_dach.to_csv('labeled_network_2022_dach_20240114.csv', index=False)